In [1]:
import pandas as pd

- USGS datasheet with 2673 earthquakes
- concat with PT_earthquake_strong to get intensity (Mercalli)

- fetch waves from all of those, select P-S arrivals and take required measures (like amplitude and period of waves)
- store in a file: X_data = eq_data (coordinates, depth, wave parameters), Y_data = mag (gathered mag)
- create a learning algorithm to predict expected magnitude based on simple parameters
- create a second algorithm to predict intensity based on distance, coordinates and expected magnitude

- extrapolate to newer arrivals...

- check how much time it takes to compute. remember that for close-earthquakes there isn't a big gap between p-phase arrival and full-blast

Códigos SEED [abc]: <br>
a - banda (E - extremely short period; S - short period; H - high broadband; B - broadband; L - long period); <br>
b - sensibilidade do sismómetro (H- alto ganho; L- baixo ganho);<br>
c - componentes (Z-Vertical; * - Vertical, Norte - Sul e Este - Oeste) <br>

In [10]:
df = pd.read_csv("usgs_2005_to_present.csv")
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2024-10-23T00:21:45.510Z,37.6361,-20.2312,10.000,4.3,mb,15.0,112.0,4.200,0.99,...,2024-10-23T12:53:07.366Z,Azores Islands region,earthquake,12.32,1.940,0.137,15.0,reviewed,us,us
1,2024-09-04T16:41:51.459Z,38.7598,-0.5454,5.000,2.9,ml,12.0,125.0,1.225,0.77,...,2024-10-11T03:40:11.040Z,"1 km SSE of Alfafara, Spain",earthquake,4.06,2.006,0.128,8.0,reviewed,us,us
2,2024-08-26T04:11:37.277Z,38.0222,-9.5047,6.586,5.0,mwr,107.0,28.0,1.038,0.56,...,2024-10-28T21:38:00.155Z,"56 km W of Sines, Portugal",earthquake,5.69,3.418,0.075,17.0,reviewed,us,us
3,2024-08-13T22:51:37.616Z,35.4614,-3.6174,10.000,4.1,mb,33.0,115.0,2.326,0.69,...,2024-10-26T15:48:09.040Z,"37 km NE of Al Hoceïma, Morocco",earthquake,4.91,1.951,0.198,7.0,reviewed,us,us
4,2024-05-27T00:40:24.609Z,35.5311,-3.5066,10.000,4.3,mb,67.0,71.0,2.379,0.58,...,2024-08-06T20:32:45.040Z,"49 km NE of Al Hoceïma, Morocco",earthquake,6.36,1.848,0.078,47.0,reviewed,us,us
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,2005-01-02T11:22:53.290Z,35.1990,-3.8500,0.000,2.9,m,6.0,294.9,NaN,NaN,...,2014-11-07T01:24:32.376Z,"5 km N of Imzouren, Morocco",earthquake,NaN,NaN,NaN,NaN,reviewed,mdd,mdd
2668,2005-01-02T06:42:04.220Z,35.0860,-3.1720,0.000,3.1,m,5.0,312.8,NaN,NaN,...,2014-11-07T01:24:32.096Z,"17 km WNW of Al Aaroui, Morocco",earthquake,NaN,NaN,NaN,NaN,reviewed,mdd,mdd
2669,2005-01-02T02:03:51.210Z,34.9220,-5.3760,30.000,3.1,m,4.0,316.4,NaN,NaN,...,2014-11-07T01:24:31.999Z,"23 km NE of Ouezzane, Morocco",earthquake,NaN,NaN,NaN,NaN,reviewed,mdd,mdd
2670,2005-01-01T22:42:37.460Z,35.4320,-4.0950,0.000,3.0,m,13.0,275.2,NaN,NaN,...,2014-11-07T01:24:31.865Z,"24 km NW of Al Hoceïma, Morocco",earthquake,NaN,NaN,NaN,NaN,reviewed,mdd,mdd


In [11]:
df2 = pd.read_csv("PT_earthquakes_strong_T.csv")
df2

,Unnamed: 0,magnitude,depth,epicenter,mercali
0,"Monday, Aug 26, 2024, at 04:11 am (GMT +0)",5.4,17.5 km,38.0067°N / 9.4651°W,"['V', 'Moderate shaking near epicenter']"
1,"Tuesday, Apr 23, 2024, at 02:40 pm (GMT -1)",5.2,18 km,38.1576°N / 30.8347°W,"['V', 'Moderate shaking near epicenter']"
2,"Thursday, Apr 4, 2024, at 12:20 pm (GMT -2)",5.0,10 km,36.1683°N / 33.9707°W,"['V', 'Moderate shaking near epicenter']"
3,"Friday, Sep 29, 2023, at 06:05 pm (GMT -1)",5.0,14.1 km,38.4809°N / 30.347°W,"['V', 'Moderate shaking near epicenter']"
4,"Tuesday, Jan 17, 2023, at 03:40 pm (GMT -2)",5.1,16.6 km,42.1313°N / 29.2871°W,"['V', 'Moderate shaking near epicenter']"
...,...,...,...,...,...
181,"Tuesday, Jul 17, 1923, at 11:06 pm (GMT -2)",5.5,10 km,43.697°N / 29.843°W,"['VI', 'Strong shaking near epicenter']"
182,"Saturday, Jun 30, 1917, at 03:20 pm (GMT -1)",5.6,15 km,38.091°N / 26.807°W,"['VI', 'Strong shaking near epicenter']"
183,"Sunday, Jul 11, 1915, at 11:28 am (GMT +0)",5.9,15 km,36.76°N / 12.577°W,"['VI', 'Strong shaking near epicenter']"
184,"Friday, Nov 25, 1910, at 01:34 am (Universal T...",5.5,10 km,35.433°N / 16.686°W,"['VI', 'Strong shaking near epicenter']"
